# Import packages

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error
import GPy
from functions.data_simulation import data_simulation7
from functions.model_training import ICM


from sklearn.cluster import SpectralClustering as sc
import numpy.random as rand
from numpy.linalg import norm as dist
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from functools import partial
from scipy.special import expit
from sklearn.linear_model import Ridge as ridge
from sklearn.linear_model import Lasso as lasso
from sklearn.linear_model import LinearRegression as ols
from sklearn.svm import SVR as kernel_svr
from sklearn.svm import LinearSVR as lin_svr
from sklearn.kernel_ridge import KernelRidge as kernel_ridge

from sklearn.tree import DecisionTreeRegressor as reg_tree
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.ensemble import AdaBoostRegressor as ada_reg
from sklearn.ensemble import GradientBoostingRegressor as gbr
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

from sklearn.model_selection import GridSearchCV

import copy

import statsmodels.api as sm
import statsmodels.formula.api as smf

rmse = lambda x,y: np.power(mse(x,y), 0.5)

import pandas as pd

import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri as np2ri

robjects.numpy2ri.activate()
import numpy as np
from sklearn.metrics import mean_squared_error
import GPy

# Define functions

In [ ]:
rfr_reg_parameter_pairs = [(rfr(),{#'max_depth':[None] + list(range(2,11,4)),
                             'min_samples_leaf':list(range(1,11,3)), 
                             'n_estimators':[100, 200]})]




linear_reg_parameter_pairs = []
linear_reg_parameter_pairs.append((ridge(), {'alpha':np.logspace(-7, 7,8),
                                             'tol':[1e-3]}))

def get_best_model_from_gridcv(X, Y, reg_parameter_pairs, print_flag = False, verbose = 0):
    val_errs = []
    models = []
    x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)
    for reg, parameters in reg_parameter_pairs:
        if print_flag:
            print('trying ', str(reg)[:50])
        model = GridSearchCV(reg, parameters, cv=2, refit=True, verbose=verbose, n_jobs = 10)
        model.fit(X,Y)
        val_errs.append(rmse(y_test, model.predict(x_test)))
#         if print_flag:
#             print('trying ' + str(reg)[:60], val_errs[-1])
        models.append(copy.deepcopy(model.best_estimator_))
    min_ind = val_errs.index(min(val_errs))
    if verbose != 0:
        print(str(models[min_ind]), 'val rmse:{}'.format(val_errs[min_ind]))
    return copy.deepcopy(models[min_ind])

def run_methodGP(X_rct, Y_rct, T_rct, X_obs, Y_obs, T_obs, _internal_X_not_rct):
    assert(X_rct.shape[0] == Y_rct.shape[0])
    assert(T_rct.shape[0] == Y_rct.shape[0])
    assert(X_obs.shape[0] == Y_obs.shape[0])
    assert(T_obs.shape[0] == Y_obs.shape[0])
    
    # Naive GP trained on the observational data
    
    # Control
    import GPy
    kern = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
    f0_gp_model = GPy.models.GPRegression(np.vstack(X_obs[T_obs==0]), np.vstack(Y_obs[T_obs==0]), kern)
    f0_gp_model.optimize()

    # Treated
    kern = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
    f1_gp_model = GPy.models.GPRegression(np.vstack(X_obs[T_obs==1]),
                                          np.vstack(Y_obs[T_obs==1]),
                                          kern)
    f1_gp_model.optimize()

    # Predict the expectation
    omega_rct_pred = f1_gp_model.predict(np.vstack(X_rct))[0] - f0_gp_model.predict(np.vstack(X_rct))[0]
    omega_obs_pred = f1_gp_model.predict(np.vstack(X_obs))[0] - f0_gp_model.predict(np.vstack(X_obs))[0]
    omega_all_pred = f1_gp_model.predict(np.vstack(X_all))[0] - f0_gp_model.predict(np.vstack(X_all))[0]
    omega_not_rct_pred = f1_gp_model.predict(np.vstack(_internal_X_not_rct))[0] - f0_gp_model.predict(np.vstack(_internal_X_not_rct))[0]
    
    
    _propensity_rct = np.sum(T_rct)/T_rct.shape[0]
    cate_rct_est = ite_adjusted_outcome(Y_rct, T_rct, _propensity=_propensity_rct) # CHECKED

    # omega  and cate_rct_est should both have a shape (n_rct,)
    #assert(cate_rct_est.shape == omega_rct_pred.shape)
    eta_rct_est = np.vstack(cate_rct_est) - np.vstack(omega_rct_pred)
    #assert(len(eta_rct_est.shape) == 1)
    
    best_eta_est_linear = get_best_model_from_gridcv(X_rct.reshape(-1,1), eta_rct_est, linear_reg_parameter_pairs)
    
    return copy.deepcopy(best_eta_est_linear), omega_obs_pred, omega_not_rct_pred


# Compare models

In [ ]:
rmse_naiveGP_exp = np.zeros((100))
rmse_naiveGP_obs = np.zeros((100))
rmse_KallusGP = np.zeros((100))
rmse_ICM = np.zeros((100))

bias_naiveGP_exp = np.zeros((100))
bias_naiveGP_obs = np.zeros((100))
bias_KallusGP = np.zeros((100))
bias_ICM = np.zeros((100))

var_naiveGP_exp = np.zeros((100))
var_naiveGP_obs = np.zeros((100))
var_KallusGP = np.zeros((100))
var_ICM = np.zeros((100))

best_rho = pd.read_csv('sim7_best_rho.csv').values[0]


for i in range(100):
    print(i+1)
    data_exp, data_obs = data_simulation7(1000, beta0=-3.0, beta1=-3.0)
    # Data preprocessing
    X0_obs = data_obs[:,1][data_obs[:,2]==0]
    X1_obs = data_obs[:,1][data_obs[:,2]==1]
    Y0_obs = data_obs[:,3][data_obs[:,2]==0]
    Y1_obs = data_obs[:,3][data_obs[:,2]==1]

    X0_exp = data_exp[:,1][data_exp[:,2]==0]
    X1_exp = data_exp[:,1][data_exp[:,2]==1]
    Y0_exp = data_exp[:,3][data_exp[:,2]==0]
    Y1_exp = data_exp[:,3][data_exp[:,2]==1]

    test_data = np.arange(-2.0,2.04,0.04)

    trueCATE = 1+test_data
    
    # Naive GP trained on the observational data
    kern = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
    m0_obs1 = GPy.models.GPRegression(np.vstack(X0_obs),np.vstack(Y0_obs),kern)
    m0_obs1.optimize()
    kern = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
    m1_obs1 = GPy.models.GPRegression(np.vstack(X1_obs),np.vstack(Y1_obs),kern)
    m1_obs1.optimize()
    # Predict the expectation
    mu0_obs1 = m0_obs1.predict(np.vstack(test_data))[0]
    mu1_obs1 = m1_obs1.predict(np.vstack(test_data))[0]
    # Derive CATE
    naiveCATE_obs1 = mu1_obs1 - mu0_obs1
    

    # Naive GP trained on the experimental data
    kern = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
    m0_exp1 = GPy.models.GPRegression(np.vstack(X0_exp),np.vstack(Y0_exp),kern)
    m0_exp1.optimize()
    kern = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
    m1_exp1 = GPy.models.GPRegression(np.vstack(X1_exp),np.vstack(Y1_exp),kern)
    m1_exp1.optimize()
    # Predict the expectation
    mu0_exp1 = m0_exp1.predict(np.vstack(test_data))[0]
    mu1_exp1 = m1_exp1.predict(np.vstack(test_data))[0]
    # Derive CATE
    naiveCATE_exp1 = mu1_exp1 - mu0_exp1

    # Kallus GP
    Y_all = np.r_[data_obs[:,3], data_exp[:,3]]
    X_all = np.r_[data_obs[:,1], data_exp[:,1]]
    T_all = np.r_[data_obs[:,2], data_exp[:,2]]
    X_eval = test_data
    _mean_outcome = np.mean(data_obs[:,3])
    _propensity_all = np.sum(data_obs[:,2])/data_obs[:,2].shape[0]
    def ite_adjusted_outcome(Y,T,_propensity, c = _mean_outcome):
        assert(_propensity>0)
        assert(_propensity<1)
        assert(len(Y.shape) == 1)
        assert(Y.shape == T.shape )
        _ite = np.zeros(Y.shape)
        _ite[T==1] =  (Y[T==1]- c)/_propensity
        _ite[T==0] = -(Y[T==0] - c)/(1-_propensity)
        return _ite


    eta_2step_modelGP, omega_obs_pred_2stepGP, omega_eval_pred_2stepGP = run_methodGP(data_exp[:,1], data_exp[:,3], 
                                                                                      data_exp[:,2], data_obs[:,1], 
                                                                                      data_obs[:,3], data_obs[:,2], 
                                                                                      test_data)

    tau_pred_eval_2stepGP = eta_2step_modelGP.predict(test_data.reshape(-1,1)) + omega_eval_pred_2stepGP


    # ICM
    m0, m1 = ICM(X_E = data_exp[:,1], X_O = data_obs[:,1], 
                 T_E = data_exp[:,2], T_O = data_obs[:,2], 
                 Y_E = data_exp[:,3], Y_O = data_obs[:,3],  
                 r = 2, ID=1, AD=0, rho=0.8)
    predY0_exp1 = m0.predict_noiseless(np.c_[np.vstack(test_data),np.ones(test_data.shape[0])*0])
    predY1_exp1 = m1.predict_noiseless(np.c_[np.vstack(test_data),np.ones(test_data.shape[0])*0])
    predCATE_exp1 = predY1_exp1[0] - predY0_exp1[0]



    rmse_ICM[i] = mean_squared_error((predCATE_exp1), (trueCATE), squared = False)
    rmse_KallusGP[i] = mean_squared_error((tau_pred_eval_2stepGP), trueCATE, squared = False)
    rmse_naiveGP_exp[i] = mean_squared_error((naiveCATE_exp1), trueCATE, squared = False)
    rmse_naiveGP_obs[i] = mean_squared_error((naiveCATE_obs1), trueCATE, squared = False)

    bias_naiveGP_exp[i] = np.mean(naiveCATE_exp1 - trueCATE)
    bias_naiveGP_obs[i] = np.mean(naiveCATE_obs1 - trueCATE)
    bias_KallusGP[i] = np.mean(tau_pred_eval_2stepGP - trueCATE)
    bias_ICM[i] = np.mean(predCATE_exp1 - trueCATE)

    var_naiveGP_exp[i] = np.mean((naiveCATE_exp1 - np.mean(naiveCATE_exp1))**2)
    var_naiveGP_obs[i] = np.mean((naiveCATE_obs1 - np.mean(naiveCATE_obs1))**2)
    var_KallusGP[i] = np.mean((tau_pred_eval_2stepGP - np.mean(tau_pred_eval_2stepGP))**2)
    var_ICM[i] = np.mean((predCATE_exp1 - np.mean(predCATE_exp1))**2)




# Save results

In [3]:
# Convert NumPy arrays to pandas DataFrames
sim7_modelComparison_RMSEs_df = pd.DataFrame(np.c_[rmse_naiveGP_exp, rmse_naiveGP_obs, rmse_KallusGP, rmse_ICM,
                                                   bias_naiveGP_exp, bias_naiveGP_obs, bias_KallusGP, bias_ICM,
                                                   var_naiveGP_exp, var_naiveGP_obs, var_KallusGP, var_ICM], 
                                                     columns=['RMSE_naiveGPexp', 'RMSE_naiveGPobs', 'RMSE_KallusGP', 'RMSE_ICM',
                                                              'bias_naiveGPexp', 'bias_naiveGPobs', 'bias_KallusGP', 'bias_ICM',
                                                              'var_naiveGPexp', 'var_naiveGPobs', 'var_KallusGP', 'var_ICM'])

# Save data_exp1 to a CSV file
sim7_modelComparison_RMSEs_df.to_csv('sim7_modelComparison.csv', index=False)